<h1>0. Packages</h1>

In [1]:
from importlib import reload
import numpy as np
import re
import skyfield.sgp4lib as spg4
import matplotlib 
import PyQt5

# User defined lib.
import generate_debris as gd
import CoordTransforms as ct

reload(gd)
reload(ct)

matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy.stats import uniform
from scipy.stats import norm
import scipy.stats as stats
from enum import IntEnum    
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from itertools import product, combinations
from matplotlib import cm
from scipy import integrate
import pandas as pd
from numba import njit, prange


%matplotlib qt5

debris_category = IntEnum('Category', 'rb sc soc')

<h1>1. Data Structure</h1>

<h3>1.1 Satellite Structure</h3>

Creating an object to represent satellite from 3le.txt <br>
May end up using implementation from another package

<b>Data sources:<b><br>
https://www.space-track.org    
https://www.celestrak.com/NORAD/documentation/spacetrk.pdf

<b>Information about 3le/2le:</b><br>
https://en.wikipedia.org/wiki/Two-line_element_set#cite_note-nasahelp-12
https://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/SSOP_Help/tle_def.html

In [2]:
class two_line_element:
    
    def __init__(self, data):
        self.parse_data(data)
        
    def parse_data(self, satelite_data):
        self.title = satelite_data[0]
        line_one = satelite_data[1].split()
        line_two = satelite_data[2].split()

        self.catalog_number = re.search('[0-9]{1,5}', line_one[0]).group()
        self.classification = re.search('[UCS]', line_one[0]).group()

        international_designator = re.search('([0-9]{2})([0-9]{3})(\w{1,3})', line_one[1])
        self.launch_year = international_designator.group(1)
        self.launch_number = international_designator.group(2)
        self.launch_piece =  international_designator.group(3)

        epoch = re.search('([0-9]{2})(\d+\.\d+)', line_one[2])
        self.epoch_year = epoch.group(1)
        self.epoch_day = epoch.group(2)

        self.ballistic_coefficient = float(line_one[3])
        self.mean_motion_double_prime = line_one[4]
        self.BSTAR = line_one[5]

        self.inclination = line_two[1]
        self.right_ascension_of_ascending_node = line_two[2]
        self.eccentricity = line_two[3]
        self.argument_of_perigee = line_two[4]

        mean_anomaly = line_two[5]

<h3>Data structure implementation using spg4 from skyfield library<h3>

In [3]:
# Opening the .txt file
with open("3le.txt") as f:
    txt = f.read()
    
sat_lines = re.findall('(.*?)\n(.*?)\n(.*?)\n', txt)

# Preview of `sat_lines`, each element should be the three lines representing a satellite
print(sat_lines[0])

# Convert each group of 3 lines into a satelite object
def line_element_to_satellite(lines):
    title = lines[0]
    line1 = lines[1]
    line2 = lines[2]
    return spg4.EarthSatellite(line1, line2, name=title)

satellites = [line_element_to_satellite(lines) for lines in sat_lines]

# Verify data structure by finding element representing the ISS
iss_index = [sat.name for sat in satellites].index("0 ISS (ZARYA)")
iss = satellites[iss_index]

('0 VANGUARD 1', '1     5U 58002B   19352.86539842  .00000184  00000-0  20948-3 0  9994', '2     5  34.2610  67.7117 1845282 255.2293  83.7559 10.84789954185624')


<h3>1.2 Celestial Bodies</h3>


de421.bsp is a Ephemerides provided by JPL Horizon which has the calculated positions of celestial bodies within a certain time interval. de421 is commonly used due to its small size and its relativley up to date information

In [4]:
from skyfield.api import load, EarthSatellite
ts = load.timescale(builtin=True)

# Loading the data from de421.bsp using skyfield 
planets = load('de421.bsp')

# Finding the data about Earth
earth = planets["Earth"]

<h3>1.3 Celestial Bodies</h3>

Plotting the satellite data in a 3D view.<br>
Accidently broke my implementation so need to start with the below base and get it to plot the ISS again.

<b>Source:</b> https://space.stackexchange.com/questions/25958/how-can-i-plot-a-satellites-orbit-in-3d-from-a-tle-using-python-and-skyfield

<h1>2. Breakup Models</h1>

<h3>2.1 NASA breakup model</h3>

<b>Implementation:</b> (Found on page 207)<br>
https://www.researchgate.net/publication/295490674_Space_debris_cloud_evolution_in_Low_Earth_Orbit

<b>Alternate implementation:</b><br>
https://gitlab.obspm.fr/apetit/nasa-breakup-model/tree/master

<b>Information:</b><br>
https://digitalcommons.calpoly.edu/cgi/viewcontent.cgi?referer=https://en.wikipedia.org/&httpsredir=1&article=1094&context=theses


<b>Now implemented in file generate_debris.py</b><br>

<h3>2.2 NASA breakup model validation</h3>

In [ ]:
# Explosion Case

reload(gd)

L_c, areas, masses, AM = gd.fragmentation(1000, 0, 0, False, debris_category.rb, True)

bins = [1e-3, 1e-2, 1e-1, 1, np.inf] # L_c bins in meters
h,b = np.histogram(L_c, bins=bins)

ch = [ { f'>{bins[i]}m':np.sum(h[i:]) for i in range(len(h))}]

df = pd.DataFrame(ch)
df

In [ ]:
# reload(gd)


# L_c = gd.characteristic_lengths(1000, 10, 10, True, debris_category.rb, False)
# N_fragments_total = L_c.shape[0]
# lambda_c = np.log10(L_c)
# areas = gd.avg_area(L_c)

# def create_log_bins(values, nbins=100):
#     #return np.geomspace(values.min(), values.max(), nbins)
#     bins = np.geomspace(values.min(), values.max(), nbins)
#     a = bins[1]/bins[0]
#     bins = np.concatenate([[bins[0]/a], bins,[bins[-1]*a]])
#     return bins

# Validating L_c
# L_c = gd.characteristic_lengths(1000, 10, 10, True, debris_category.rb, False)
# h, b = np.histogram(L_c, bins=create_log_bins(L_c))
# plt.figure(figsize=(5,5))
# plt.xscale('log')
# plt.xlabel(r'$L_{c}$')
# plt.ylabel(r'$N_f$')
# plt.plot((b[:-1] + b[1:])/2, h, '.-')
# plt.show()
# np.mean(counts)


# Validating Areas
# h, b = np.histogram(areas, bins=create_log_bins(areas))
# plt.figure(figsize=(5,5))
# plt.xscale('log')
# plt.xlabel(r'$Areas$')
# plt.ylabel(r'$N_f$')
# plt.plot((b[:-1] + b[1:])/2, h, '.-')
# plt.show()

# I = np.argwhere(b > 10e-2)
# count = np.sum(h[I - 1])
# print(count)
# # Validating Mass & Velocity
# AM = np.array(gd.distribution_AM(lambda_c, gd.debris_category.rb))
# masses = areas / 10**AM

# h, b = np.histogram(masses, bins=create_log_bins(masses))
# plt.figure(figsize=(5,5))
# plt.xscale('log')
# plt.xlabel(r'$Masses$')
# plt.ylabel(r'$N_f$')
# plt.gca().set_yticks([0, 2, 4, 6, 8])
# plt.gca().set_xticks([1e-8, 1e-5, 1e-2, 10, 1e4])
# plt.gca().set_xlim([1e-8,1e4])
# plt.gca().set_ylim([0,8])
# plt.grid()
# plt.plot((b[:-1] + b[1:])/2, h/1e5, '.-')
# plt.show()

# # Validating Velocity
# deltaV = np.array(gd.distribution_deltaV(AM, 10, True))


# h, b = np.histogram(deltaV, bins=create_log_bins(deltaV))
# plt.figure(figsize=(5,5))
# plt.xscale('log')
# plt.xlabel(r'$\delta V$')
# plt.ylabel(r'$N_f$')
# plt.gca().set_yticks([0, 2, 4, 6, 8])
# plt.gca().set_xticks([1e-3, 1e-2, 1e-1, 1, 10])
# plt.gca().set_xlim([1e-3,10])
# plt.gca().set_ylim([0,8])
# plt.grid()
# plt.plot((b[:-1] + b[1:])/2, h/1e5, '.-')
# plt.show()

<h3>2.3 and Onward will be alternate breakup models</h3>

<h1>3. Numerical Propagation</h1>

<h3>3.1 Modeling the Atmosphere</h3>

In [5]:
excel_file = "AtmosphericModelValues.xlsx"
earth_radius = 6378.0 #km
tabulated_values = pd.read_excel(excel_file, engine="openpyxl")

def atmosphere_density(altitude):
    
    bins = tabulated_values['Altitude Lower Bound (km)'].values
    base_altitude = tabulated_values['Base Altitude (km)'].values
    nominal_density = tabulated_values['Nominal Density (kg/m^3)'].values
    scale_height = tabulated_values['Scale Height (km)'].values
    i = np.digitize(altitude, bins) - 1
    return (nominal_density[i]*np.exp(-(altitude-base_altitude[i])/(scale_height[i])))

def scale_height(altitude):
    bins = tabulated_values['Altitude Lower Bound (km)'].values
    scale_height = tabulated_values['Scale Height (km)'].values
    i = np.digitize(altitude, bins) - 1
    return scale_height[i]


<h3>3.2 Calculating Orbital Debris Properties</h3>

In [6]:
from skyfield.api import load, EarthSatellite

ts = load.timescale(builtin=True)

t = ts.now()
geocentric = iss.at(t)
init_position = geocentric.position.km

iss_mass = 41970 # kg (reduced by an order of mag for sim purposes)
projectile_mass  = 227 # kg

L_c, areas, masses, AM = gd.fragmentation(iss_mass, projectile_mass, 5, False, debris_category.soc, False)
deltaV = np.array(gd.distribution_deltaV(AM, 10, True))

<h3>3.3 Converting Cartesian to Keplerian</h3>

In [7]:
from numpy.linalg import norm

grav_param_earth = 398600.4418 #km^3s^-2
init_position = geocentric.position.km

deb_positions = np.empty((len(AM), 3))
deb_positions[:, :] = init_position[None,:]
deb_velocities = gd.velocity_vectors(len(AM), geocentric.velocity.km_per_s, deltaV)

keplerian_state = ct.rv2coe(deb_positions, deb_velocities, grav_param_earth)

In [ ]:
rand_index = 0
print("Cartesian")
print("r: ", deb_positions[rand_index, :])
print("v: ", deb_velocities[rand_index, :])

print("Keplerian")
ks_rand = keplerian_state[:, rand_index]
print("a: ", ks_rand[0])
print("e: ", ks_rand[1])
print("i: ", ks_rand[2])
print("Omega: ", ks_rand[3])
print("omega: ", ks_rand[4])
print("M: ", ks_rand[5])
print("nu: ", ks_rand[6])
print()

<h3>3.4 Testing coverting back to Cartesian</h3>

In [ ]:
reload(ct)

indexes = np.random.default_rng().choice(keplerian_state.shape[1], size=1000, replace=False)
ks_sample = keplerian_state[:, indexes]

cartesian_state = np.array(ct.coe2rv_many(k=grav_param_earth,
                              p=ks_sample[7, :],
                              ecc=ks_sample[1, :],
                              inc=ks_sample[2, :],
                              raan=ks_sample[3, :],
                              argp=ks_sample[4, :],
                              nu=ks_sample[6, :]))

r = cartesian_state[0, :, :]
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(r[:, 0], r[:, 1], r[:, 2])
ax.set_xlim3d(-6378.0-1000, 6378.0+1000)
ax.set_xlabel('X (km)')

ax.set_ylim3d([-6378.0-1000, 6378.0+1000])
ax.set_ylabel('Y (km)')

ax.set_zlim3d([-6378.0-1000, 6378.0+1000])
ax.set_zlabel('Z (km)')

earth_radius = 3378.0 #km
_u, _v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
_x = earth_radius*np.cos(_u)*np.sin(_v)
_y = earth_radius*np.sin(_u)*np.sin(_v)
_z = earth_radius*np.cos(_v)

ax.plot_surface(_x, _y, _z, cmap=cm.coolwarm)
plt.show()

<h3>3.4 Debris Cloud formation</h3>

In [8]:
## Shrink number of debris being used
indexes = np.random.default_rng().choice(keplerian_state.shape[1], size=250, replace=False)

ks = keplerian_state[:, indexes]
masses = masses[indexes]
areas = areas[indexes]
deb_positions = deb_positions[indexes, :]
deb_velocities = deb_velocities[indexes, :]

In [9]:
np.savez("debris.npz",
         keplerian_states=ks,
         masses=masses,
         areas=areas,
         deb_positions=deb_positions,
         deb_velocities=deb_velocities)

In [10]:
container = np.load('debris.npz')
ks = container['keplerian_states']
masses = container['masses']
areas = container['areas']
deb_positions = container['deb_positions']
deb_velocities = container['deb_velocities']

<h4>3.4.1 Ellisoid formation</h4>

In [80]:
from numba import jit 
from numpy import arccos as acos
from numpy import cos, dot, sin, sqrt
from numpy.linalg import norm
from scipy.constants import pi

# Modified from OrbitalPy.utilities
@njit(parallel=True, fastmath=True)
def M2E(e_deb, M_t, tolerance=1e-14):
    """Convert mean anomaly to eccentric anomaly.
    Implemented from [A Practical Method for Solving the Kepler Equation][1]
    by Marc A. Murison from the U.S. Naval Observatory
    [1]: http://murison.alpheratz.net/dynamics/twobody/KeplerIterations_summary.pdf
    """
    n_deb = M_t.shape[0]
    n_times = M_t.shape[1]
    
    E_t = np.empty_like(M_t)
    
    for i in prange(n_deb):
        e = e_deb[i]
        for j in prange(n_times):
            M = M_t[i, j]
            
            MAX_ITERATIONS = 100
            Mnorm = np.mod(M, 2 * pi)
            E0 = M + (-1 / 2 * e ** 3 + e + (e ** 2 + 3 / 2 * cos(M) * e ** 3) * cos(M)) * sin(M)
            dE = tolerance + 1
            count = 0
            while dE > tolerance:
                t1 = cos(E0)
                t2 = -1 + e * t1
                t3 = sin(E0)
                t4 = e * t3
                t5 = -E0 + t4 + Mnorm
                t6 = t5 / (1 / 2 * t5 * t4 / t2 + t2)
                E = E0 - t5 / ((1 / 2 * t3 - 1 / 6 * t1 * t6) * e * t6 + t2)
                dE = np.abs(E - E0)
                E0 = E
                count += 1
                if count == MAX_ITERATIONS:
                    print('Did not converge, increase number of iterations')
            E_t[i, j] = E
    return E_t


In [77]:
ks[1].shape

(250,)

In [118]:
import gc

# Doing for the 1 orbit, using the period of the slowest deb.
mu = 398600.4418 #km^3s^-2
T = ks[8, :]

t1, t2, dt = 0, 20*np.ceil(max(T)), 60*5

times      = np.arange(t1, t2, dt)

# Mean anomaly rate of change
M_dt       = np.sqrt(mu/ks[0, :]**3)

Nd         = len(M_dt)
Nt         = len(times)

# Mean anomaly over time
M_t        = np.deg2rad(ks[5, :, None]) + M_dt[:, None] * times[None, :]
M_t        = np.rad2deg(np.mod(M_t, 2*np.pi))

# Eccentric anomaly over time. Note need to use E_t in rad, thus convert to deg after using it in
# x1 and x2
E_t        = np.empty(shape=(Nd, Nt), dtype=np.float32)
E_t        = M2E(ks[1], np.deg2rad(M_t))

x1         = np.sqrt(1 + ks[1, :])[:, None] * np.sin(E_t / 2)
x2         = np.sqrt(1 - ks[1, :])[:, None] * np.cos(E_t / 2)
E_t        = np.rad2deg(E_t)

# True anomaly over time
nu_t       = (2*np.arctan2(x1, x2) % (2*np.pi))
nu_t       = np.rad2deg(nu_t).T

In [119]:
import visualize_deb as vis
reload(vis)

# Remove data for debris that have a periapsis less than or equal to earh radius
periapsis = ks[0, :] * (1 - ks[1, :])
indexes = np.argwhere(periapsis > 6371)

ks_pruned = np.squeeze(ks[:, indexes])
nu_pruned = nu_t[:, indexes][:, :, 0]

cartesian_states = np.empty(shape=(len(times), 2, ks_pruned.T.shape[0], 3))
for i in prange(len(times)):
    cartesian_state = np.array(ct.coe2rv_many(k=mu,
                                              p=ks_pruned[7, :],
                                              ecc=ks_pruned[1, :],
                                              inc=ks_pruned[2, :],
                                              raan=ks_pruned[3, :],
                                              argp=ks_pruned[4, :],
                                              nu=nu_pruned[i, :])
                              )
    cartesian_states[i] = cartesian_state
                                        
data = np.swapaxes(cartesian_states,0,1)[0, :, :, :]
#vis.generate_visualization(data, "Ellipsoid2")


<h4>3.4.2 Particle Debris flux</h4>

Using a particle flux to determine when the fragments of the debris have finished the formation of the ring. Indicating the end of the first phase of the debris cloud formation. This is accomplished by creating an xz plane and detecting when particles have switched from one side to the other. This approach will cause a peak as fragments pass through that becomes uniform as the debris becomes uniformly spread out.

In [125]:
def fragmentation_flux(X):
    return np.sum((X[:-1, :, 1] < 0) & (X[1:, :, 1] > 0), axis=1)
    
    

In [201]:
position = cartesian_states[:, 0, :, :]
flux = fragmentation_flux(position)
peaks, peak_plateaus = find_peaks(- flux, plateau_size = 1)

test_flux = flux[:100]

plt.plot(range(len(test_flux)), test_flux)
plt.show()

def converged(X, tol):
    n = 10
    diff = X[-n-1: -1] - X[-n-2: -2]
    mean = np.mean(abs(diff))
    return mean < tol
    
    
converged(test_flux, 0.02 * position.shape[1])

True

In [188]:
position.shape[1] * 0.01

2.29

In [130]:
plt.plot(range(len(flux)), np.gradient(flux))
plt.show()

In [105]:
import mpl_toolkits.mplot3d.axes3d as p3

fig = plt.figure()
ax = p3.Axes3D(fig)

ax.set_xlim3d(-6378.0-1000, 6378.0+1000)
ax.set_xlabel('X (km)')

ax.set_ylim3d([-6378.0-1000, 6378.0+1000])
ax.set_ylabel('Y (km)')

ax.set_zlim3d([-6378.0-1000, 6378.0+1000])
ax.set_zlabel('Z (km)')

# Plot central body
earth_radius = 3378.0 #km
_u, _v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
_x = earth_radius*np.cos(_u)*np.sin(_v)
_y = earth_radius*np.sin(_u)*np.sin(_v)
_z = earth_radius*np.cos(_v)

ax.plot_surface(_x, _y, _z, cmap=cm.coolwarm)

_u, _v = np.mgrid[0:2*np.pi/3:20j, 0:np.pi:10j]
_x = 1.5*earth_radius*np.cos(_u)*np.sin(_v)
_y = 0 * _u
_z = 1.1*earth_radius*np.cos(_v)

ax.plot_surface(_x, _y, _z, cmap=cm.seismic)
plt.show()

<h3> Implementing drag </h3>

"where cd is the drag coefficient of the fragment, which is assumed to be constant and equal to 2.2 (Valdo)"

In [ ]:
from scipy.special import iv


radius_earth = 6371 # km 
def deda_dt(t, state, A, M):    
    
    e, a = state.copy().reshape((2, len(A))) # Needed to be copied why?
    
    I = a*(1-e) < radius_earth + 50 # 50 km above earth sats are doomed 
    a[I] = 0
    e[I] = 0
    
    """Returns differential equation modeling drag on each piece of debris
    Args:
        M (array): Mass
        e (array): Eccentricity
        A (array): Cross-sectionl Area
        a (array): Semi-Major Axis
        r (array): Position
    """
    c_d = 2.2 # Drag coefficient
    drag_coef = (-(c_d * A) / M)
    
    atm_density = atmosphere_density(a - radius_earth)
    #atm_density[np.argwhere(np.isnan(atm_density))] = 0
    
    # dedt
    dedt = np.zeros_like(e)
    I    = (e>=0.001)
    dedt[I] = (drag_coef[I] * np.sqrt(grav_param_earth / a[I]) * atm_density[I])
    I    = I & (e<0.01)
    x    = (a[I] * e[I]) / scale_height(a[I])
    dedt[I] *= iv(1,x) + (e[I]/2)*(iv(0,x) + iv(2,x))
    
    # dadt
    dadt = np.empty_like(a)                          
    dadt = drag_coef * np.sqrt(grav_param_earth * a) * atm_density

    I    = (e>=0.001) & (e < 0.01)
    x    = (a[I] * e[I]) / scale_height(a[I])
    dadt[I] *= (iv(0,x) + 2*e[I]*iv(1,x))
    I = (e >= 0.01)
    x    = (a[I] * e[I]) / scale_height(a[I])    
    dadt[I] *= iv(0, x) + 2*e[I]*iv(1, x) + (3/4)*e[I]**2*(iv(0, x) + iv(2, x)) + (e[I]**3/4)*(3*iv(1, x) + iv(3, x))
    
    dedt[np.argwhere(np.isnan(dedt))] = 0
    dadt[np.argwhere(np.isnan(dadt))] = 0
    
    output = np.hstack((dedt, dadt))
    
    return output

In [ ]:
state = np.concatenate((keplerian_states[1, :], keplerian_states[0, :]))


In [ ]:
times = np.linspace(0, T[0]*5, 2)
output = integrate.solve_ivp(deda_dt, times, state, t_eval = times, args = (areas, masses), vectorized=True)

In [ ]:
de = output.y[0:len(areas), :]
da = output.y[len(areas):output.y.shape[0], :]

In [ ]:
print(keplerian_states[1,0])
print(de[0, :])

In [ ]:
periapsis = keplerian_states[0, :] * (1 - keplerian_states[1, :])
print( np.sum(periapsis < 6378) / np.sum(periapsis >= 6378))

# NEED TO ASK ABOUT INTEGRATION AND ARRAY FRIENDLY CONVERSION
<h3>3.6 Performing the integration </h3>

<h1>4. Visualization </h1>